In [2]:
# Guessing this is where we will be saving the extrcated features, but not very sure yet. Let's follow along the notebook, and see where it leads us to.

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from data_generator import DataGenerator, batch_predict

2025-08-28 15:03:35.979914: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-28 15:03:37.572879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-28 15:03:37.572942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-28 15:03:37.725719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 15:03:38.029068: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# load the model?

In [5]:
overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping.keras") 

2025-08-28 15:05:53.919503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [6]:
overlapping_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [7]:
# locate the test data?

In [8]:
# input features
input_features_loc = 'data/test_input_sub_images/'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [9]:
input_contents

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [9]:
# Running inot a lot of memory issues, therefore the loop is cut into chunks to get and save the required outputs.

In [10]:
%%time
for_sanity_check = []
for i in range(6, len(input_contents)):
    # load the np file
    load_np_file = np.load(os.path.join(input_features_loc, input_contents[i]))
    # print shape of the loaded file
    print(load_np_file.shape)
    # predicted_values
    # define the test data generator here
    test_gen = DataGenerator(feature1, target, batch_size=32, shuffle=False)
    predicted_values = overlapping_model.predict(load_np_file)
    print(predicted_values.shape)
    for_sanity_check.append(predicted_values)
    # save these values?
    # name
    loc_name = 'data/predicted_sequences_from_stage_1/' + 'pred_values_blk_' + input_contents[i].split('.')[0][-4:] + '.npy'
    np.save(loc_name, predicted_values)

In [11]:
# might need to do a batch predict due to the dataset size, let's figure this out - okay, why don't we actually do this and try to write a cleaner code? That might be better for future refernces.

In [12]:
# Nah let it be for now, it would take some time - we could get all the extracted features saved in two runs, I think it's okay for now

In [13]:
# Okay, we have done another sanity check here, which i believe is not necessary, but let's try it anyway until we get a memory error

In [11]:
# perform a sanity check
loc_path = 'data/predicted_sequences_from_stage_1/'

In [12]:
loc_contents = os.listdir(loc_path)
loc_contents.sort()

In [13]:
loc_contents

['pred_values_blk_0103.npy',
 'pred_values_blk_0104.npy',
 'pred_values_blk_0105.npy',
 'pred_values_blk_0106.npy',
 'pred_values_blk_0201.npy',
 'pred_values_blk_0202.npy',
 'pred_values_blk_0205.npy',
 'pred_values_blk_0206.npy',
 'pred_values_blk_0302.npy',
 'pred_values_blk_0303.npy',
 'pred_values_blk_0304.npy',
 'pred_values_blk_0305.npy',
 'pred_values_blk_0306.npy']

In [14]:
# Okay, for this we do need the entire loop to run to get the predicted values - so let's just do the sanity check starting from the 6th file here

In [22]:
%%time
for i in range(6, len(loc_contents)):
    load_stored_preds = np.load(os.path.join(loc_path, loc_contents[i]))
    print(np.mean(load_stored_preds == for_sanity_check[i-6]))

In [23]:
pred_values = np.load(os.path.join(loc_path, loc_contents[0]))

In [24]:
pred_values.shape

(3072, 7, 32)

In [25]:
# I think then we have examined the extracted features against the true values - so let's continue this work after food?

In [27]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [28]:
out_contents

['test_targets_blk_0103.npy',
 'test_targets_blk_0104.npy',
 'test_targets_blk_0105.npy',
 'test_targets_blk_0106.npy',
 'test_targets_blk_0201.npy',
 'test_targets_blk_0202.npy',
 'test_targets_blk_0205.npy',
 'test_targets_blk_0206.npy',
 'test_targets_blk_0302.npy',
 'test_targets_blk_0303.npy',
 'test_targets_blk_0304.npy',
 'test_targets_blk_0305.npy',
 'test_targets_blk_0306.npy']

In [29]:
# just do this to one npy file
true_targets_blk_0103 = np.load(os.path.join(out_targets_loc, out_contents[0]))

In [30]:
true_targets_blk_0103.shape

(3072, 7, 32)

In [31]:
test_in_0_true = true_targets_blk_0103[0,:,:]

In [32]:
test_in_0_true.shape

(7, 32)

In [33]:
test_in_0_true_df = pd.DataFrame(test_in_0_true)

In [34]:
test_in_0_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.989890,0.0,1.820586,0.0,0.0,...,0.0,0.0,2.039670,0.0,2.116731,0.0,0.0,2.093492,1.995151,0.0
1,0.0,0.0,0.0,0.0,0.0,1.800317,0.0,1.655207,0.0,0.0,...,0.0,0.0,1.819944,0.0,1.878130,0.0,0.0,1.872040,1.765169,0.0
2,0.0,0.0,0.0,0.0,0.0,0.123075,0.0,0.109234,0.0,0.0,...,0.0,0.0,0.133679,0.0,0.145644,0.0,0.0,0.113552,0.108494,0.0
3,0.0,0.0,0.0,0.0,0.0,1.217194,0.0,1.115794,0.0,0.0,...,0.0,0.0,1.244642,0.0,1.294032,0.0,0.0,1.280463,1.213219,0.0
4,0.0,0.0,0.0,0.0,0.0,2.876129,0.0,2.639863,0.0,0.0,...,0.0,0.0,2.952369,0.0,3.044252,0.0,0.0,3.023037,2.874696,0.0


In [35]:
# Okay, these values seem much more closer to the values we had when we had the 300,300,3 windows

In [36]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(test_in_0_true_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.989890,0.0,1.820586,0.0,0.0,0.946177,0.0,0.0,2.374387,0.0,0.0,0.0,2.413269,0.0,2.339803,2.113288,0.0,0.0,0.0,2.039670,0.0,2.116731,0.0,0.0,2.093492,1.995151,0.0
1,0.0,0.0,0.0,0.0,0.0,1.800317,0.0,1.655207,0.0,0.0,0.804122,0.0,0.0,2.079869,0.0,0.0,0.0,2.198049,0.0,2.085574,1.903789,0.0,0.0,0.0,1.819944,0.0,1.878130,0.0,0.0,1.872040,1.765169,0.0
2,0.0,0.0,0.0,0.0,0.0,0.123075,0.0,0.109234,0.0,0.0,0.000000,0.0,0.0,0.143035,0.0,0.0,0.0,0.164116,0.0,0.145700,0.142227,0.0,0.0,0.0,0.133679,0.0,0.145644,0.0,0.0,0.113552,0.108494,0.0
3,0.0,0.0,0.0,0.0,0.0,1.217194,0.0,1.115794,0.0,0.0,0.548958,0.0,0.0,1.440583,0.0,0.0,0.0,1.516132,0.0,1.448876,1.310991,0.0,0.0,0.0,1.244642,0.0,1.294032,0.0,0.0,1.280463,1.213219,0.0
4,0.0,0.0,0.0,0.0,0.0,2.876129,0.0,2.639863,0.0,0.0,1.444209,0.0,0.0,3.398966,0.0,0.0,0.0,3.510002,0.0,3.367321,3.066525,0.0,0.0,0.0,2.952369,0.0,3.044252,0.0,0.0,3.023037,2.874696,0.0
5,0.0,0.0,0.0,0.0,0.0,1.557331,0.0,1.420637,0.0,0.0,0.696551,0.0,0.0,1.846516,0.0,0.0,0.0,1.904458,0.0,1.841774,1.654445,0.0,0.0,0.0,1.599984,0.0,1.647395,0.0,0.0,1.631798,1.563028,0.0
6,0.0,0.0,0.0,0.0,0.0,0.593198,0.0,0.564214,0.0,0.0,0.210090,0.0,0.0,0.699687,0.0,0.0,0.0,0.783950,0.0,0.726200,0.654302,0.0,0.0,0.0,0.606471,0.0,0.651529,0.0,0.0,0.646013,0.594337,0.0


In [37]:
# Let' not worry too much about the all 0 columns - this is not the case for all dfs, just a few

In [38]:
zero_cols = (test_in_0_true_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 21


In [39]:
# Okay, now what?

In [40]:
# So we do have the extracted features for the test images in each sequence. Now we need the true features for the train images in the sequence for the BLAR model as well. I think that's what we will do next?

In [41]:
# where is this model?
fine_tuned_model = tf.keras.models.load_model("../../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5")

In [42]:
fine_tuned_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [43]:
# Define the feature extractor model

# feature extractor input
feat_ext_input = fine_tuned_model.input

# feature extractor output 
feat_ext_output = fine_tuned_model.layers[-4].output

# define the model
feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [44]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [49]:
# Okay - now what do we need to do?

# I think we do have the inputs stored and arranged in a previous exercise, may be we can use these?

# Where is this location?

sub_windows_of_images_loc = 'data/test_input_sub_images/'

In [52]:
contents_here = os.listdir(sub_windows_of_images_loc)
contents_here.sort()

In [53]:
contents_here

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [54]:
# Let's just try this for a single block, and maybe write a function so that it could be done for the rest?

In [55]:
load_blk_0103_data = np.load(os.path.join(sub_windows_of_images_loc, contents_here[0]))

In [56]:
load_blk_0103_data.shape

(3072, 13, 100, 100, 3)

In [57]:
time_1 = load_blk_0103_data[:,0,:,:,:]

In [58]:
time_1.shape

(3072, 100, 100, 3)

In [59]:
# I think we can get preds for this?
extracted_features_t1 = feature_extractor_model.predict(time_1)

2025-08-28 15:20:12.469751: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


96/96 [==============================] - 3s 4ms/step


In [60]:
extracted_features_t1.shape

(3072, 32)

In [61]:
# just convert this to a df to verify something
extracted_features_t1_df = pd.DataFrame((extracted_features_t1))

In [62]:
extracted_features_t1_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.040617,0.0,0.961652,0.0,0.0,...,0.0,0.0,1.070542,0.0,1.116127,0.0,0.0,1.084534,1.030262,0.0
1,0.0,0.0,0.0,0.0,0.0,1.652722,0.0,1.522594,0.0,0.0,...,0.0,0.0,1.692298,0.0,1.763141,0.0,0.0,1.718137,1.638624,0.0
2,0.0,0.0,0.0,0.0,0.0,1.575529,0.0,1.452407,0.0,0.0,...,0.0,0.0,1.617116,0.0,1.680444,0.0,0.0,1.638901,1.560891,0.0
3,0.0,0.0,0.0,0.0,0.0,1.027982,0.0,0.939284,0.0,0.0,...,0.0,0.0,1.056456,0.0,1.100152,0.0,0.0,1.077257,1.027676,0.0
4,0.0,0.0,0.0,0.0,0.0,0.402013,0.0,0.366982,0.0,0.0,...,0.0,0.0,0.421354,0.0,0.453295,0.0,0.0,0.427051,0.407254,0.0


In [63]:
extracted_features_t1_df.shape

(3072, 32)

In [64]:
# cool, I think now we can go ahead and view all the columns in the dataframe?

In [66]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(extracted_features_t1_df.head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.040617,0.0,0.961652,0.0,0.0,0.398845,0.0,0.0,1.236845,0.0,0.0,0.0,1.289144,0.0,1.219131,1.114823,0.0,0.0,0.0,1.070542,0.0,1.116127,0.0,0.0,1.084534,1.030262,0.0
1,0.0,0.0,0.0,0.0,0.0,1.652722,0.0,1.522594,0.0,0.0,0.725593,0.0,0.0,1.961552,0.0,0.0,0.0,2.034237,0.0,1.928091,1.757249,0.0,0.0,0.0,1.692298,0.0,1.763141,0.0,0.0,1.718137,1.638624,0.0
2,0.0,0.0,0.0,0.0,0.0,1.575529,0.0,1.452407,0.0,0.0,0.691447,0.0,0.0,1.871270,0.0,0.0,0.0,1.941004,0.0,1.842280,1.679118,0.0,0.0,0.0,1.617116,0.0,1.680444,0.0,0.0,1.638901,1.560891,0.0
3,0.0,0.0,0.0,0.0,0.0,1.027982,0.0,0.939284,0.0,0.0,0.394355,0.0,0.0,1.203080,0.0,0.0,0.0,1.284402,0.0,1.216868,1.098629,0.0,0.0,0.0,1.056456,0.0,1.100152,0.0,0.0,1.077257,1.027676,0.0
4,0.0,0.0,0.0,0.0,0.0,0.402013,0.0,0.366982,0.0,0.0,0.071219,0.0,0.0,0.462750,0.0,0.0,0.0,0.535250,0.0,0.497594,0.446137,0.0,0.0,0.0,0.421354,0.0,0.453295,0.0,0.0,0.427051,0.407254,0.0


In [67]:
# but notice that over here we do not have all zero columns - make sense we have not concatenated data in a time direction yet
zero_cols = (extracted_features_t1_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 0


In [68]:
# see, there are any columns will all zeros, so we should be all okay for the BLAR model

In [71]:
# load_blk_0103_data.shape[1]

In [72]:
# Let's get the predictions across all time points in a for loop?
catch_all_preds_block_0103 = []
for i in range(load_blk_0103_data.shape[1]):
    time_wise_data = load_blk_0103_data[:,i,:,:,:]
    extracted_features = feature_extractor_model.predict(time_wise_data)
    catch_all_preds_block_0103.append(extracted_features)

96/96 [==============================] - 0s 4ms/step


In [73]:
# check the shape of the prediction arrays?

In [74]:
for pred_array in catch_all_preds_block_0103:
    print(pred_array.shape)

(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)
(3072, 32)


In [75]:
# stack all these together? - maybe to be of shape 910, 13, 32
stacked_features_0103 = np.stack(catch_all_preds_block_0103, axis = 1)

In [76]:
stacked_features_0103.shape

(3072, 13, 32)

In [77]:
sub_0 = stacked_features_0103[0,:,:]

In [78]:
sub_0_df = pd.DataFrame(sub_0)

In [80]:
sub_0_df.shape

(13, 32)

In [81]:
with pd.option_context('display.max_columns', None):
    display(sub_0_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.040617,0.0,0.961652,0.0,0.0,0.398845,0.0,0.0,1.236845,0.0,0.0,0.0,1.289144,0.0,1.219131,1.114823,0.0,0.0,0.0,1.070542,0.0,1.116127,0.0,0.0,1.084534,1.030262,0.0
1,0.0,0.0,0.0,0.0,0.0,0.517676,0.0,0.475709,0.0,0.0,0.118368,0.0,0.0,0.606281,0.0,0.0,0.0,0.637740,0.0,0.591632,0.549053,0.0,0.0,0.0,0.526487,0.0,0.552225,0.0,0.0,0.538950,0.504612,0.0
2,0.0,0.0,0.0,0.0,0.0,1.506203,0.0,1.378870,0.0,0.0,0.658157,0.0,0.0,1.788515,0.0,0.0,0.0,1.843211,0.0,1.749850,1.590830,0.0,0.0,0.0,1.539276,0.0,1.611196,0.0,0.0,1.554219,1.477109,0.0
3,0.0,0.0,0.0,0.0,0.0,0.558141,0.0,0.503735,0.0,0.0,0.134302,0.0,0.0,0.654528,0.0,0.0,0.0,0.693614,0.0,0.648710,0.594727,0.0,0.0,0.0,0.573714,0.0,0.600081,0.0,0.0,0.561909,0.531341,0.0
4,0.0,0.0,0.0,0.0,0.0,1.665863,0.0,1.529454,0.0,0.0,0.750836,0.0,0.0,1.990490,0.0,0.0,0.0,2.044719,0.0,1.946582,1.777378,0.0,0.0,0.0,1.705272,0.0,1.772821,0.0,0.0,1.745788,1.664704,0.0
5,0.0,0.0,0.0,0.0,0.0,0.411679,0.0,0.378218,0.0,0.0,0.094445,0.0,0.0,0.494226,0.0,0.0,0.0,0.540134,0.0,0.502141,0.451709,0.0,0.0,0.0,0.425089,0.0,0.468233,0.0,0.0,0.431291,0.391923,0.0
6,0.0,0.0,0.0,0.0,0.0,1.072753,0.0,0.985232,0.0,0.0,0.435253,0.0,0.0,1.247671,0.0,0.0,0.0,1.351402,0.0,1.272865,1.153844,0.0,0.0,0.0,1.101992,0.0,1.143098,0.0,0.0,1.112941,1.060320,0.0
7,0.0,0.0,0.0,0.0,0.0,2.679819,0.0,2.456954,0.0,0.0,1.321609,0.0,0.0,3.192454,0.0,0.0,0.0,3.263349,0.0,3.125001,2.851504,0.0,0.0,0.0,2.753466,0.0,2.830257,0.0,0.0,2.814368,2.684098,0.0
8,0.0,0.0,0.0,0.0,0.0,2.460829,0.0,2.281760,0.0,0.0,1.230245,0.0,0.0,2.941373,0.0,0.0,0.0,3.022441,0.0,2.872968,2.626414,0.0,0.0,0.0,2.528879,0.0,2.611843,0.0,0.0,2.584795,2.462703,0.0
9,0.0,0.0,0.0,0.0,0.0,0.372708,0.0,0.337889,0.0,0.0,0.029252,0.0,0.0,0.433929,0.0,0.0,0.0,0.471211,0.0,0.437511,0.403790,0.0,0.0,0.0,0.381727,0.0,0.408376,0.0,0.0,0.389570,0.364560,0.0


In [82]:
zero_cols = (sub_0_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 21


In [83]:
# There are all 0 columns - but is this true for all sub windows?

In [84]:
sub_98 = stacked_features_0103[98,:,:]

In [85]:
sub_98_df = pd.DataFrame(sub_98)

In [86]:
with pd.option_context('display.max_columns', None):
    display(sub_98_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.000000,0.0,0.000000,0.216274,0.0,0.225201,0.0,0.000000,0.000000,0.0,0.000000,0.268049,0.000000,0.0,0.000000,0.309067,0.0,0.278625,0.258152,0.0,0.000000,0.000000,0.224079,0.0,0.269482,0.000000,0.000000,0.245853,0.212209,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.143880,0.0,0.136291,0.0,0.000000,0.000000,0.0,0.000000,0.161935,0.000000,0.0,0.000000,0.209916,0.0,0.192846,0.172560,0.0,0.000000,0.000000,0.140635,0.0,0.172047,0.000000,0.000000,0.145673,0.132263,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.124316,0.0,0.119178,0.0,0.000000,0.000000,0.0,0.000000,0.147583,0.000000,0.0,0.000000,0.176351,0.0,0.162334,0.140141,0.0,0.000000,0.000000,0.133984,0.0,0.153980,0.000000,0.000000,0.128703,0.116672,0.0
3,0.0,0.0,0.017213,0.0,0.000000,0.054474,0.0,0.043679,0.0,0.000000,0.000000,0.0,0.000000,0.052252,0.050581,0.0,0.000000,0.073706,0.0,0.071652,0.059800,0.0,0.000000,0.015079,0.054327,0.0,0.065518,0.000000,0.000000,0.041666,0.043076,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.107011,0.0,0.101606,0.0,0.000000,0.000000,0.0,0.000000,0.114587,0.000000,0.0,0.000000,0.156071,0.0,0.136753,0.129740,0.0,0.000000,0.000000,0.107581,0.0,0.130816,0.000000,0.000000,0.105331,0.093591,0.0
5,0.0,0.0,0.047746,0.0,0.012689,0.033869,0.0,0.017933,0.0,0.004491,0.000000,0.0,0.000000,0.009796,0.081622,0.0,0.000000,0.031104,0.0,0.035700,0.030200,0.0,0.000000,0.049343,0.029039,0.0,0.035881,0.002540,0.025294,0.017103,0.018935,0.0
6,0.0,0.0,0.000000,0.0,0.000000,0.415214,0.0,0.382448,0.0,0.000000,0.077154,0.0,0.000000,0.478336,0.000000,0.0,0.000000,0.544377,0.0,0.499203,0.456280,0.0,0.000000,0.000000,0.424293,0.0,0.451962,0.000000,0.000000,0.427795,0.406813,0.0
7,0.0,0.0,0.000000,0.0,0.000000,0.162419,0.0,0.171750,0.0,0.000000,0.000000,0.0,0.000000,0.190274,0.000000,0.0,0.000000,0.266189,0.0,0.215954,0.204757,0.0,0.000000,0.000000,0.162199,0.0,0.195943,0.000000,0.000000,0.193320,0.170838,0.0
8,0.0,0.0,0.000000,0.0,0.000000,0.167096,0.0,0.160692,0.0,0.000000,0.000000,0.0,0.000000,0.207961,0.000000,0.0,0.000000,0.250526,0.0,0.202127,0.193699,0.0,0.000000,0.000000,0.176961,0.0,0.188651,0.000000,0.000000,0.169999,0.166220,0.0
9,0.0,0.0,0.101138,0.0,0.059977,0.002155,0.0,0.005521,0.0,0.054057,0.000000,0.0,0.051298,0.000000,0.123545,0.0,0.000646,0.010046,0.0,0.000000,0.011075,0.0,0.032983,0.108542,0.000000,0.0,0.011228,0.057101,0.073509,0.000000,0.000000,0.0


In [87]:
# Are there any 0 all columns?
zero_cols = (sub_98_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 11


In [88]:
stacked_features_0103.shape

(3072, 13, 32)

In [89]:
train_save_path = 'data/train_features_non_overlapping/'

In [90]:
np.save(os.path.join(train_save_path, 'train_features_block_0103.npy'), stacked_features_0103)

In [91]:
san_check_blk_0103 = np.load('data/train_features_non_overlapping/train_features_block_0103.npy')

In [92]:
np.mean(stacked_features_0103 == san_check_blk_0103)

1.0

In [93]:
# Okay, so let's define a function for this
sub_windows_of_images_loc

'data/test_input_sub_images/'

In [94]:
train_save_path

'data/train_features_non_overlapping/'

In [95]:
def store_train_extracted_features(path_to_inputs, input_feature_file, save_path):
    # load the file
    loaded_input_file = np.load(os.path.join(path_to_inputs, input_feature_file))
    # Let's get the predictions across all time points in a for loop?
    catch_all_preds = []
    for i in range(loaded_input_file.shape[1]):
        time_wise_data = loaded_input_file[:,i,:,:,:]
        extracted_features = feature_extractor_model.predict(time_wise_data)
        catch_all_preds.append(extracted_features)

    # stack these predictions?
    stacked_features = np.stack(catch_all_preds, axis = 1)
    # save the stack of extracted features?
    save_name = 'train_features_block_' + input_feature_file.split('.')[0][-4:] + '.npy'
    np.save(os.path.join(save_path, save_name), stacked_features)
    # also do the sanity check?
    print(np.mean(np.load(os.path.join(save_path, save_name)) == stacked_features))
    return stacked_features
    

In [96]:
# see if this works for block 0103?
stack_0103 = store_train_extracted_features(sub_windows_of_images_loc, contents_here[0], train_save_path)

96/96 [==============================] - 0s 3ms/step
1.0


In [97]:
# Seems to be working, do this to the rest of the blocks?

In [98]:
%%time
# Easier to do it in a for loop - but verify this tomorrow
all_stacks = []
for i in range(len(contents_here)):
    stack = store_train_extracted_features(sub_windows_of_images_loc, contents_here[i], train_save_path)
    all_stacks.append(stack)
    

96/96 [==============================] - 0s 3ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 5ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
96/96 [==============================] - 0s 4ms/step
1.0
CPU times: user 2min 38s, sys: 1min 50s, total: 4min 28s
Wall time: 5min 16s
